# Creating Redshift Serverless using AWS python SDK

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-serverless.html

#unhash if haven't installed yet!
#pip install boto3

In [ ]:
import pandas as pd
import boto3
import json

# Setting up - Attaching your  AWS secret and access key

- Once have taken note of the access key and secret from your IAM user
- Create a config file in the same folder as this notebook and fill
<font color='red'>
<BR>
[AWS]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>
<font/>

In [ ]:
##Load DWH Params from your config file
#You should load in your key, secret, name of namespace, workgroup, database, user, and IAM role as well as database password, port and base capacity for redshift.

import configparser

config = configparser.ConfigParser()
configFilePath = r'C:\Users\The user\Documents\course\IntegralyticCourse\DataEngineering\RedshiftProject\configuration.cfg'
config.read(configFilePath)
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')
DWH_NAMESPACE_NAME = config.get('DWH', 'DWH_NAMESPACE_NAME')
DWH_WORKGROUP_NAME = config.get('DWH', 'DWH_WORKGROUP_NAME')
DWH_BASE_CAPACITY = config.get('DWH', 'DWH_BASE_CAPACITY')
DWH_DB = config.get('DWH', 'DWH_DB')
DWH_DB_USER = config.get('DWH', 'DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH', 'DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH', 'DWH_PORT')
DWH_IAM_ROLE_NAME = config.get('DWH', 'DWH_IAM_ROLE_NAME')
REGION = config.get('AWS', 'REGION')


#here is an example of a dataframe you can print out with all your params, you can take out password if you dont want it to print
pd.DataFrame({"Param":
                  [ "ACCESS_KEY", "SECRET_KEY", "DWH_BASE_CAPACITY", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_NAMESPACE_NAME, DWH_WORKGROUP_NAME, DWH_BASE_CAPACITY, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

Create clients for IAM and RedshiftServerless

In [ ]:
######Your code here######
redshift_client = boto3.client('redshift-serverless', 
                      aws_access_key_id = KEY,
                      aws_secret_access_key = SECRET,
                      region_name = REGION)

iam_client = boto3.client('iam', 
                      aws_access_key_id = KEY,
                      aws_secret_access_key = SECRET,
                      region_name = REGION)

## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam/client/create_role.html
for AssumeRolePolicyDocument, open the role you created through the console under trust relationship, trusted entities, use that json for that perameter. See image 1


In [ ]:
# TODO: Create the IAM role
print("1.1 Creating a new IAM Role")


######Your code here######
DwhRole = iam_client.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description='Allows Redshift clusters to call AWS services on your behalf.',
        AssumeRolePolicyDocument=json.dumps({
              "Version": "2012-10-17",
              "Statement": [
                {
                  "Effect": "Allow",
                  "Principal": {
                    "Service": "redshift.amazonaws.com"
                  },
                  "Action": "sts:AssumeRole"
                }
              ]
            }),
    )

### STEP 1.2: Attach Policy to IAM ROLE
- attach policy to the role
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam/client/attach_role_policy.html
for PolicyArn see image 2


In [ ]:
# TODO: Attach Policy
print("1.2 Attaching Policy")



######Your code here######

iam_client.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']



### STEP 1.3: Get IAM role ARN
- get the IAM role ARN and write it to your config file, so you can use to load data from S3 to Redshift using the ARN
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam/client/get_role.html

In [ ]:
# TODO: Get and print the IAM role ARN, save it to your config file
print("1.3 Get the IAM role ARN")



######Your code here######

roleArn = iam_client.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)


## Step 2 Create A Redshift Serverless Instance

### Step 2.1 Create A Redshift Serverless Namespace

for complete arguments to create_namespace see
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-serverless/client/create_namespace.html

In [ ]:
response = redshift_client.create_namespace(
    adminUserPassword = DWH_DB_PASSWORD,
    adminUsername = DWH_DB_USER,
    dbName= DWH_DB,
    defaultIamRoleArn= roleArn,
    iamRoles=[
        roleArn,
    ],
    logExports=[
        'userlog'
    ],
    namespaceName= DWH_NAMESPACE_NAME
)

### Step 2.2 Create A Redshift Serverless Workgroup
 https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-serverless/client/create_workgroup.html

In [ ]:
######Your code here######

response = redshift_client.create_workgroup(
    baseCapacity=int(DWH_BASE_CAPACITY),
    namespaceName=DWH_NAMESPACE_NAME,
    publiclyAccessible=True,
    workgroupName=DWH_WORKGROUP_NAME
)


### Step 2.3 Check if your workgroup is available, get the host address and write it to your config file
Additionally, you can take the response object and make it into a nice dataframe with relevant and useful information about your workgroup

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-serverless/client/create_workgroup.html

In [ ]:
######Your code here######

response = redshift_client.get_workgroup(
    workgroupName=DWH_WORKGROUP_NAME
)

### Step 2.4 Creating a security group. 


In the console...

## Step 3 Test
Connect to your database and run a test query

In [ ]:
import psycopg2


conn = psycopg2.connect(database = DWH_DB,
                        user = DWH_DB_USER,
                        host= 'my-redshift-workgroup-1.940788980600.us-east-1.redshift-serverless.amazonaws.com',
                        password = DWH_DB_PASSWORD,
                        port = "5439")

cur = conn.cursor()


In [ ]:
cur.execute("select 1")

In [ ]:
conn.commit()
conn.close()